# Decision Trees

Decision trees are versitile ML aglorithms that can perform classification, regression and multioutput tasks. Decision trees are also fundamentla components of Random Forests, which are among the most powerful ML aglorithms today.

We will discuss how to train, visualize and make predictions. then we will go through CART trainng algorithm used by Scikit-Learn, and we will discuss how to regularise trees and use them for regression tasks. We will also discuss the limitations

Lets bult a decision tree using a DecisionTreeClassifier on the iris dataset

In [2]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()
X = iris.data[:, 2:] # Petal length and width
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X, y)

DecisionTreeClassifier(max_depth=2)

In [3]:
# we use the export_graphviz method to output a graph definition file called iris_tree.dot

from sklearn.tree import export_graphviz

export_graphviz(
    tree_clf,
    out_file="iris_tree.dot",
    feature_names=iris.feature_names[2:],
    class_names=iris.target_names,
    rounded=True,
    filled=True
)

# then we can use the dot command line tool from Graphviz package ti convert .dot image to png/pdf etc.
# dot -Tpng iris_tree.dot -o iris_tree.png


<img src="./iris_tree.png">

Equation for gini impurity:

$G_i=1-\sum_{k=1}^{n}p_{i,k}^2$

Where $p_{i,k}$ is the ratio of class k instances among the training instances in the $i^{th}$ node

For example for depth-2 left node has a gini score equal to $1-(0/54)^2-(49/54)^2-(5/54)^2 = 0.168$

The Decision Tree can also esitmate the probability that an instance belong sto a particulaar class k. First it traverses the tree to din the lead node for this instance, and then it returns the ratio of training instances of class k in this node. For example, suppose you have found a flower whose petals are 5cm long and 1.5 cm wide. The corresponding leaf node is the depth-2 left node, so decision Tree should output the following probabilites: 0% (0/54) for Iris setosa, 90.7% for Iris versicolor (49/54) and 9.3% for Iris verginica (5/54). Let's check this 

In [4]:
tree_clf.predict_proba([[5, 1.5]])

array([[0.        , 0.90740741, 0.09259259]])

## CART Training Algorithm

Scikit-Learn uses the Classification and Regression Tree (CART) algorithm to train Decision Trees. The algorithm works by first splitting the training set into two subsets using a single feature k and a threshold $t_k$ (e.g., "petal length $\leq$ 2.5cm") How does it chooes k and $t_k$? It searches for the pair (k, $t_k$) that produces the purest subsets (weighted by their size).

Cost function that CART algorithm tries to minimise:

J(k,$t_k$) = $\dfrac{m_{left}}{m}G_{left} + \dfrac{m_{right}}{m}G_{right}$

Where $G_{left/right}$ measures the impurity of the left/right subset<br>
and $m_{left_right}$ measures the number of instances in the left/right subset 

once it has successfully split the training set into 2, it keeps slitting using the same logic. It stops recusing once it reaches the maximum depth or if it cannot find a split that will reduce impurity. As you can tell, this algorithm is a greedy algorithm - it greedily searches for an optimum split at the top level and repeats the process at each subsequent level. It does not check whether or not the split will lead to the lowest possible impurity several levels down, often greedy algorithms like this finds a solution that's reasonably good but not optimum.

Unfortunately, finding the optimal tree is known to be an NP-Complete problem: it requires O(EXP(m)) time, that's why we have to settle for reasonably good algorithms

## Regularization Hyperparameters

Decision Trees make very few assumptions about the training data ( as opposed to linear models, which assume that the data is linear ). If left unconstrained, the tree structure will adapt itself to the training data, most likely overfitting it. Such a model is often called a `nonparametric model`, not because it does not have any hyperparameters, but because the number of parameters is not determined prior to training. In contrast, a `parametric model` such as a linear model, has a predetermined number of parameters, so its degree of freedom is limited, reducing the risk of overfitting (but increasing the risk of underfitting).

To solve this, we can use the `max_depth` hyperparameter (default is None, so means unlimited). It also has other parameters that similarly restict the shape of the Decision Tree: min_samples_split, mon_samples_lead, min_weight_fration_leaf, min_samples_leaf, max_lead_nodes, max_features. Increasing min_* hyperparameters or reducing max_* hyperparameters will regularise the model.